<a href="https://colab.research.google.com/github/JH-code-lab/Data-Analysis-with-Open-Source/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4%EA%B8%B0%EB%B0%98%20%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D_%EC%A4%91%EA%B0%84%EA%B3%BC%EC%A0%9C%EB%AC%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
from datetime import datetime

api_key = '72c8d37f3dd90e1f959648cbae9231f08541f91c528bd7e7109c806b855dcd58'

start = datetime(2015, 1, 1)
end = datetime(2024, 12, 1)

dates = []
while start <= end:
    dates.append(start.strftime("%Y%m"))
    if start.month == 12:
        start = datetime(start.year + 1, 1, 1)
    else:
        start = datetime(start.year, start.month + 1, 1)


for ym in dates:
    # URL 안에 searchDate 쿼리값을 월별로 변경
    url = f'http://apis.data.go.kr/1613000/ApHusEnergyUseInfoOfferServiceV2/getWntyAvrgEnergyUseAmountInfoSearchV2?serviceKey={api_key}&searchDate={ym}'

    response = requests.get(url)

    if response.status_code == 200:
        print(response.json())
    else:
        print(f"{ym} API 호출 실패: {response.status_code}")

print("api 호출성공")




{'response': {'body': {'item': {'heat': 532, 'waterHot': 132, 'gas': 5, 'elect': 674, 'waterCool': 176}}, 'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}}}
{'response': {'body': {'item': {'heat': 435, 'waterHot': 126, 'gas': 5, 'elect': 639, 'waterCool': 174}}, 'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}}}
{'response': {'body': {'item': {'heat': 329, 'waterHot': 123, 'gas': 4, 'elect': 532, 'waterCool': 173}}, 'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}}}
{'response': {'body': {'item': {'heat': 157, 'waterHot': 106, 'gas': 3, 'elect': 554, 'waterCool': 180}}, 'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}}}
{'response': {'body': {'item': {'heat': 51, 'waterHot': 88, 'gas': 2, 'elect': 506, 'waterCool': 184}}, 'header': {'resultCode': '00', 'resultMsg': 'NORMAL SERVICE.'}}}
{'response': {'body': {'item': {'heat': 29, 'waterHot': 63, 'gas': 2, 'elect': 539, 'waterCool': 198}}, 'header': {'resultCode': '00', 'resultMsg':

In [3]:
import requests
import json
import time

def get_year_months():
    """2015년 1월부터 2024년 12월까지의 년월 리스트를 생성하는 함수"""
    year_months = []
    for year in range(2015, 2024+1):
        for month in range(1, 12+1):
            year_months.append(f'{year}{month:02d}')
    return year_months

def collect_energy_data():
    """API를 호출하여 에너지 사용금액 데이터를 수집하는 함수"""
    api_key = '72c8d37f3dd90e1f959648cbae9231f08541f91c528bd7e7109c806b855dcd58'
    responses = []

    # 년월 리스트 가져오기
    year_months = get_year_months()

    print(f"총 {len(year_months)}개월의 데이터를 수집합니다...")
    print("데이터 수집을 시작합니다.\n")

    for i, year_month in enumerate(year_months, 1):
        # API URL 구성
        url = f'http://apis.data.go.kr/1613000/ApHusEnergyUseInfoOfferServiceV2/getWntyAvrgEnergyUseAmountInfoSearchV2?serviceKey={api_key}&searchDate={year_month}'

        try:
            # API 호출
            response = requests.get(url)

            # HTTP 상태 코드 확인
            if response.status_code == 200:
                # JSON 데이터 파싱
                data = response.json()
                responses.append({
                    'year_month': year_month,
                    'data': data,
                    'status': 'success'
                })
                print(f"[{i:3d}/{len(year_months)}] {year_month[:4]}년 {year_month[4:]}월 데이터 수집 완료")
            else:
                print(f"[{i:3d}/{len(year_months)}] {year_month[:4]}년 {year_month[4:]}월 데이터 수집 실패 - HTTP {response.status_code}")
                responses.append({
                    'year_month': year_month,
                    'data': None,
                    'status': 'failed',
                    'error': f'HTTP {response.status_code}'
                })

        except Exception as e:
            print(f"[{i:3d}/{len(year_months)}] {year_month[:4]}년 {year_month[4:]}월 데이터 수집 중 오류: {str(e)}")
            responses.append({
                'year_month': year_month,
                'data': None,
                'status': 'error',
                'error': str(e)
            })

        # API 호출 간격 조절 (서버 부하 방지)
        time.sleep(0.1)

    return responses

def display_results(responses):
    """수집된 데이터의 결과를 출력하는 함수"""
    print("\n" + "="*60)
    print("데이터 수집 결과 요약")
    print("="*60)

    success_count = len([r for r in responses if r['status'] == 'success'])
    failed_count = len([r for r in responses if r['status'] != 'success'])

    print(f"총 수집 시도: {len(responses)}개월")
    print(f"성공: {success_count}개월")
    print(f"실패: {failed_count}개월")

    if success_count > 0:
        print("\n성공한 데이터 샘플 (최근 3개월):")
        success_responses = [r for r in responses if r['status'] == 'success']

        for response in success_responses[-3:]:  # 최근 3개월만 표시
            year_month = response['year_month']
            year = year_month[:4]
            month = year_month[4:]
            print(f"\n--- {year}년 {month}월 데이터 ---")

            # API 응답 데이터 구조에 따라 적절히 출력
            data = response['data']
            if isinstance(data, dict):
                # 주요 필드만 출력
                if 'response' in data:
                    resp = data['response']
                    if 'header' in resp:
                        print(f"응답 코드: {resp['header'].get('resultCode', 'N/A')}")
                        print(f"응답 메시지: {resp['header'].get('resultMsg', 'N/A')}")
                    if 'body' in resp and 'items' in resp['body']:
                        items = resp['body']['items']
                        if isinstance(items, list) and len(items) > 0:
                            print(f"데이터 항목 수: {len(items)}개")
                            # 첫 번째 항목의 일부 필드 출력
                            first_item = items[0]
                            for key, value in list(first_item.items())[:3]:  # 처음 3개 필드만
                                print(f"  {key}: {value}")
                else:
                    print("데이터 구조를 확인중...")

    if failed_count > 0:
        print(f"\n실패한 요청:")
        failed_responses = [r for r in responses if r['status'] != 'success']
        for response in failed_responses[:5]:  # 처음 5개만 표시
            year_month = response['year_month']
            year = year_month[:4]
            month = year_month[4:]
            print(f"  {year}년 {month}월: {response.get('error', '알 수 없는 오류')}")

def save_data_to_file(responses, filename='energy_data.json'):
    """수집된 데이터를 JSON 파일로 저장하는 함수"""
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(responses, f, ensure_ascii=False, indent=2)
        print(f"\n데이터가 '{filename}' 파일로 저장되었습니다.")
    except Exception as e:
        print(f"\n파일 저장 중 오류 발생: {str(e)}")

# 메인 실행 부분
if __name__ == "__main__":
    print("전국 평균 에너지 사용금액 데이터 수집 프로그램")
    print("="*60)

    # 데이터 수집 실행
    responses = collect_energy_data()

    # 결과 출력
    display_results(responses)

    # 데이터 파일로 저장
    save_data_to_file(responses)

    print("\n프로그램 실행이 완료되었습니다!")

전국 평균 에너지 사용금액 데이터 수집 프로그램
총 120개월의 데이터를 수집합니다...
데이터 수집을 시작합니다.

[  1/120] 2015년 01월 데이터 수집 완료
[  2/120] 2015년 02월 데이터 수집 완료
[  3/120] 2015년 03월 데이터 수집 완료
[  4/120] 2015년 04월 데이터 수집 완료
[  5/120] 2015년 05월 데이터 수집 완료
[  6/120] 2015년 06월 데이터 수집 완료
[  7/120] 2015년 07월 데이터 수집 완료
[  8/120] 2015년 08월 데이터 수집 완료
[  9/120] 2015년 09월 데이터 수집 완료
[ 10/120] 2015년 10월 데이터 수집 완료
[ 11/120] 2015년 11월 데이터 수집 완료
[ 12/120] 2015년 12월 데이터 수집 완료
[ 13/120] 2016년 01월 데이터 수집 완료
[ 14/120] 2016년 02월 데이터 수집 완료
[ 15/120] 2016년 03월 데이터 수집 완료
[ 16/120] 2016년 04월 데이터 수집 완료
[ 17/120] 2016년 05월 데이터 수집 완료
[ 18/120] 2016년 06월 데이터 수집 완료
[ 19/120] 2016년 07월 데이터 수집 완료
[ 20/120] 2016년 08월 데이터 수집 완료
[ 21/120] 2016년 09월 데이터 수집 완료
[ 22/120] 2016년 10월 데이터 수집 완료
[ 23/120] 2016년 11월 데이터 수집 완료
[ 24/120] 2016년 12월 데이터 수집 완료
[ 25/120] 2017년 01월 데이터 수집 완료
[ 26/120] 2017년 02월 데이터 수집 완료
[ 27/120] 2017년 03월 데이터 수집 완료
[ 28/120] 2017년 04월 데이터 수집 완료
[ 29/120] 2017년 05월 데이터 수집 완료
[ 30/120] 2017년 06월 데이터 수집 완료
[ 31/120] 2017년 07월 데이터 수집 완료
[ 32